# Prepare data

In [1]:
import pandas as pd

train_data = pd.read_pickle("kaggle_data/train_data.pkl")
test_data = pd.read_pickle("kaggle_data/test_data.pkl")

# data transform and predict

## TFIDF with 2000 smaples

### decision tree

In [42]:
TFIDF_train_data = train_data.sample(n=5000)
#TFIDF_train_data = train_data

In [43]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
TFIDF_vectorizer = CountVectorizer(max_features=1000, tokenizer=nltk.word_tokenize)
TFIDF_vectorizer.fit(TFIDF_train_data['text'])

C:\Users\user\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(max_features=1000,
                tokenizer=<function word_tokenize at 0x0000025E524B2550>)

In [44]:
import os
from scipy import sparse

X_train = TFIDF_vectorizer.transform(TFIDF_train_data['text'])
Y_train = TFIDF_train_data['emotion']

'''
if os.path.isfile('kaggle_data/TFIDF_X_train.npz'):
    X_train = sparse.load_npz('kaggle_data/TFIDF_X_train.npz')
else:
    X_train = TFIDF_vectorizer.transform(train_data['text'])
    sparse.save_npz("kaggle_data/TFIDF_X_train.npz", X_train)
    
Y_train = train_data['emotion']
'''
 
if os.path.isfile('kaggle_data/TFIDF_X_test.npz'):
    X_test = sparse.load_npz('kaggle_data/TFIDF_X_test.npz')
else:
    X_test = TFIDF_vectorizer.transform(test_data['text'])
    sparse.save_npz("kaggle_data/TFIDF_X_test.npz", X_test)

In [45]:
X_train

<5000x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 69664 stored elements in Compressed Sparse Row format>

In [46]:
Y_train

21898               joy
798090          sadness
124952             fear
732730     anticipation
1417691         sadness
               ...     
1142229             joy
903034          disgust
890074              joy
159487              joy
969246     anticipation
Name: emotion, Length: 5000, dtype: object

In [47]:
X_test

<411972x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 6793321 stored elements in Compressed Sparse Row format>

In [48]:
from sklearn.tree import DecisionTreeClassifier
DT_model = DecisionTreeClassifier(random_state=1)

DT_model = DT_model.fit(X_train, Y_train)

Y_train_pred = DT_model.predict(X_train)
Y_test_pred = DT_model.predict(X_test)

## so we get the pred result
Y_train_pred

array(['joy', 'sadness', 'fear', ..., 'joy', 'joy', 'anticipation'],
      dtype=object)

In [49]:
## accuracy
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_true=Y_train, y_pred=Y_train_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 1.0


In [50]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=Y_train, y_pred=Y_train_pred) 
print(cm)

[[ 146    0    0    0    0    0    0    0]
 [   0  800    0    0    1    0    0    0]
 [   0    1  449    0    1    0    0    0]
 [   0    1    0  205    0    0    0    0]
 [   0    1    0    0 1816    0    0    0]
 [   0    1    0    0    1  700    0    0]
 [   0    0    0    0    0    0  159    0]
 [   0    0    0    0    3    0    0  715]]


### deep learning model

# result to csv

In [51]:
result_csv = pd.DataFrame(columns=['id', 'emotion'])

In [52]:
test_data

,hashtags,tweet_id,text,identification
2,[bibleverse],0x28b412,"Confident of your obedience, I write to you, k...",test
4,[],0x2de201,"""Trust is not the same as faith. A friend is s...",test
9,"[materialism, money, possessions]",0x218443,When do you have enough ? When are you satisfi...,test
30,"[GodsPlan, GodsWork]",0x2939d5,"God woke you up, now chase the day #GodsPlan #...",test
33,[],0x26289a,"In these tough times, who do YOU turn to as yo...",test
...,...,...,...,...
1867525,[],0x2913b4,"""For this is the message that ye heard from th...",test
1867529,[],0x2a980e,"""There is a lad here, which hath five barley l...",test
1867530,"[mixedfeeling, butimTHATperson]",0x316b80,When you buy the last 2 tickets remaining for ...,test
1867531,[],0x29d0cb,I swear all this hard work gone pay off one da...,test


In [53]:
result_csv['id'] = test_data['tweet_id']
result_csv

,id,emotion
2,0x28b412,NaN
4,0x2de201,NaN
9,0x218443,NaN
30,0x2939d5,NaN
33,0x26289a,NaN
...,...,...
1867525,0x2913b4,NaN
1867529,0x2a980e,NaN
1867530,0x316b80,NaN
1867531,0x29d0cb,NaN


In [54]:
result_csv['emotion'] = Y_test_pred
result_csv

,id,emotion
2,0x28b412,trust
4,0x2de201,fear
9,0x218443,sadness
30,0x2939d5,joy
33,0x26289a,anticipation
...,...,...
1867525,0x2913b4,anticipation
1867529,0x2a980e,joy
1867530,0x316b80,anger
1867531,0x29d0cb,anticipation


In [55]:
result_csv.to_csv("kaggle_data/result.csv", index=False)